In [ ]:
!pip install openai langchain pypdf chromadb tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.6 MB/s e

In [ ]:
import openai

openai.api_key = your_key

In [ ]:
system_template = """Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}

Begin!
----------------
Question: {question}
Helpful Answer:"""


In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

prompt = ChatPromptTemplate.from_messages(messages)

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("sadhana paper.pdf")
data = loader.load()

In [ ]:
'''from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
docs = text_splitter.split_documents(data)'''

'from langchain.text_splitter import RecursiveCharacterTextSplitter\n\ntext_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)\ndocs = text_splitter.split_documents(data)'

In [ ]:
# For checking
# Create Document objects for each text chunk
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
texts = text_splitter.split_text(data[1].page_content)
docs_one = [Document(page_content=t) for t in texts[:]]

In [ ]:
print(docs_one)

[Document(page_content='sequence ^w¼fw1;w2; ::;wng.^wis derived based on Bayes\nrule as in equation ( 1).\n^w¼argmax p ðxÞ¼argmaxpðxjwÞpðwÞ\npðxÞð1Þ\nwhere, p(x|w) represents the acoustic model, p(w) is the\nprior probability of the word sequence and it represents the\nlanguage model, p(x) is a constant over w.\nConventional ASR models have several demerits such\nas being speaker-dependent, recognizing isolated words,and based on small vocabulary, which makes them unvi-able for a wide range of users. The conventional ASRmodels are superseded by the recent end-to-end deeplearning-based speech recognition models, which greatlysimplify the complexity of traditional speech recognition\n[5]. The proposed work ideally put forward a novel\napproach using a deep sequential neural network-basedend-to-end multilingual low resource Indian languagespeech recognition with an improved spell correctormodule.\nThe development of a multilingual low resource Indian', metadata={}), Document(page_content=

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embedding=OpenAIEmbeddings(openai_api_key = openai.api_key)
vectorstore = Chroma.from_documents(documents=docs_one,embedding=embedding)

In [ ]:
from langchain.chat_models import ChatOpenAI

llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613",openai_api_key = openai.api_key)

In [ ]:
question = "What are the languages considered"
'''docs = vectorstore.similarity_search(question)
len(docs)'''

'docs = vectorstore.similarity_search(question)\nlen(docs)'

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs = {"prompt": prompt})
result = qa_chain({"query": question})

In [ ]:
print(result)

{'query': 'What are the languages considered', 'result': 'The languages considered in the context are Indonesian, Tibetan, four Ethiopian languages, Hindi, Tamil, Telugu, Malayalam, Kannada, Bengali, Gujarati, and Marathi.'}
